# Ingestion des données en base de données

Pour résumer, l'objectif du projet est de s'entraîner en SQL et en Python pour se maintenir à jour en SQL (notamment les CTEs, les windows functions et les RANKS) et en Python (dataframes, SQLAlchemy, Pydantic, FastAPI, tests unitaires, Mock si pertinent, et si pertinent un peu d'intégration de modèles de machine learning).

La première chose dans ce notebook ça va être de préparer les données et de les importer dans une DB PostgreSQL.


## Nettoyer les données

D'abord on regarde les données pour comprendre la structure.

In [ ]:
import pandas as pd
import numpy as np

dataset_path = "Olympic_Swimming_Results_1912to2020"

# Lire le fichier CSV et afficher les colonnes
df = pd.read_csv(dataset_path + ".csv")
print(df.columns)
print(df.head())

distinct_distances = df['Distance (in meters)'].unique()  # Assurez-vous que le nom de la colonne est correct
print("Distances distinctes:", distinct_distances)

distinct_results = df['Results'].unique()
print("Nombre de résultats distincts:", len(distinct_results))

# Sélectionner aléatoirement 100 résultats distincts (ou moins s'il y en a moins de 100)
sample_size = min(100, len(distinct_results))
random_sample = np.random.choice(distinct_results, size=sample_size, replace=False)

print(f"\n{sample_size} résultats distincts aléatoires:")
for result in random_sample:
    print(result)

On renomme les colonnes pour les rendre plus faciles à utiliser.

In [ ]:
import pandas as pd
import re

# Renommer les colonnes
df = df.rename(columns={
    'Location': 'location',
    'Year': 'year',
    'Distance (in meters)': 'distance',
    'Stroke': 'stroke',
    'Relay?': 'is_relay',
    'Gender': 'gender',
    'Team': 'team',
    'Athlete': 'athlete',
    'Results': 'results',
    'Rank': 'rank'
})

print(df.head())

On convertit la colonne is_relay en booléen.

In [ ]:
# Convertir 'is_relay' en booléen
df['is_relay'] = df['is_relay'].map(lambda x: True if x == 1 else False)

print(df.head())

On extrait la distance et le nombre de relais. On aura besoin de la distance en entier pour analyser correctement les données. Dans le CSV de base c'est un string au format [0-9]+m si c'est pas du relais (e.g 100m), et [0-9]+x[0-9]+m si c'est un relais (e.g 4x100m).

In [ ]:
import re

# Fonction pour extraire la distance et le nombre de relais
def extract_distance_and_relay(distance_str):
    if 'x' in distance_str:
        nb_relay_str, distance_str = re.findall(r'\d+', distance_str)
        return int(distance_str), int(nb_relay_str)
    else:
        distance = int(re.findall(r'\d+', distance_str)[0])
        return distance, None

# Appliquer la transformation à la colonne 'distance'
df['distance'], df['nb_relay'] = zip(*df['distance'].apply(extract_distance_and_relay))

print(df.head())

df_relay_true = df[df['is_relay'] == True]
print(df_relay_true.head())

Ici, faut harmoniser les formats de temps. Dans le CSV de base, les temps sont dans des formats divers, parfois avec des minutes et des secondes, parfois avec des heures, parfois avec des décimales. On va tout convertir en secondes, sous forme de float avec une précision à la microseconde.

In [ ]:
import re
import numpy as np

def clean_time_str(time_str):
    if isinstance(time_str, str):
        # Extraire les chiffres et les séparateurs pertinents
        cleaned = re.match(r'^(\d+:?\d*:?\d*\.?\d*)', time_str)
        if cleaned:
            return cleaned.group(1)
        elif not re.search(r'\d', time_str):
            return time_str  # Retourner la chaîne si elle ne contient aucun chiffre
    return time_str

def convert_to_seconds(time_str):
    original_value = time_str
    time_str = clean_time_str(time_str)
    
    if isinstance(time_str, float):
        return time_str, None  # Déjà en secondes
    elif isinstance(time_str, str):
        if ':' in time_str:
            # Format 00:04:37.510000
            time_parts = time_str.split(':')
            if len(time_parts) == 3:
                hours, minutes, seconds = time_parts
                total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
            else:
                minutes, seconds = time_parts
                total_seconds = int(minutes) * 60 + float(seconds)
        elif re.match(r'^\d+(\.\d+)?$', time_str):
            # Format 59.720
            total_seconds = float(time_str)
        else:
            return np.nan, original_value  # Cas de disqualification ou format invalide
    else:
        return np.nan, str(original_value)  # Cas où le type n'est pas reconnu
    
    return round(total_seconds, 3), None  # Arrondir à 3 décimales pour la précision milliseconde

# Appliquer la conversion à la colonne 'results'
df['results'], df['quit_reason'] = zip(*df['results'].apply(convert_to_seconds))

# Afficher les premières lignes pour vérification
print(df.head())
print("\nColonnes du DataFrame:", df.columns)

# Afficher 50 valeurs aléatoires du DataFrame
random_sample = df.sample(n=50, random_state=1)  # random_state pour la reproductibilité
print("\n50 valeurs aléatoires du DataFrame :")
print(random_sample)

Maintenant on va traiter le nom des athlètes.

Il y a non seulement des noms nuls (qu'on va renommer "Unknown"), mais aussi des noms multiples quand il y a des relais.

Dans le dernier cas, on va créer un tuple par athlète.

In [ ]:
df['athlete'] = df['athlete'].fillna("Unknown")

relay_rows = []

# Itérer sur chaque ligne du DataFrame
for index, row in df.iterrows():
    if row['is_relay']:
        athletes = row['athlete'].split(',')  # Séparer les noms des athlètes
        for athlete in athletes:
            relay_rows.append({
                'id': len(relay_rows) + 1,  # ID autoincrémenté
                'location': row['location'],
                'year': row['year'],
                'distance': row['distance'],
                'stroke': row['stroke'],
                'is_relay': row['is_relay'],
                'gender': row['gender'],
                'team': row['team'],
                'athlete': athlete.strip(),  # Enlever les espaces
                'results': row['results'],
                'rank': row['rank'],
                'nb_relay': row['nb_relay']
            })
    else:
        relay_rows.append({
            'id': len(relay_rows) + 1,
            'location': row['location'],
            'year': row['year'],
            'distance': row['distance'],
            'stroke': row['stroke'],
            'is_relay': row['is_relay'],
            'gender': row['gender'],
            'team': row['team'],
            'athlete': row['athlete'],
            'results': row['results'],
            'rank': row['rank'],
            'nb_relay': row['nb_relay']
        })

# Créer un nouveau DataFrame à partir des lignes de relais
df_relay_expanded = pd.DataFrame(relay_rows)

print(df_relay_expanded.head())

Pour clarifier le dataframe, voici les différentes colonnes:

- location : ville où a eu lieu le tournoi

- year : année du tournoi

- is_relay : si c'est un relais ou non

- distance : distance en mètres de la course (soit la course entière si ce n'est pas un relais, soit la distance de nage pour chaque athlète dans le cas d'un relais)

- nb_relay : nombre de relais (None si ce n'est pas un relais)

- stroke : type de nage

- gender : genre

- team : équipe

- athlete : nom de l'athlète (Unknown si inconnu)

- results : temps réalisé en secondes (float), avec une précision de la microseconde, None si disqualifié

- quit_reason : raison de la disqualification (None si pas de disqualification)

- rank : classement (0 si disqualifié, 1 si or, 2 si argent, 3 si bronze, 4 si pas de médaille, 5 si pas de donnée)

- id : on s'en sert pour former les équipes quand c'est un relais, on ne l'utilise que pour l'ingestion des données en base de données c'est pas important


## Ingérer les données en base de données

On commence par connecter la base de données, les variables d'environnement sont stockées dans un fichier .env

In [ ]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

load_dotenv()

username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")
# Création de la base de données
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{dbname}') 

On va découper les données en cinq objets:
- Athlete : id (primary key), name, gender
- Event : id (primary key), location, year, distance, stroke, is_relay, nb_relay
- NationalTeam : id (primary key), code
- EventTeam : id (primary key), event_id (foreign key), athlete_id (foreign key), national_team_id (foreign key)
- Result : id (primary key), team_id (foreign key), results, rank, quit_reason

Ainsi, on normalise les données

In [ ]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean, ForeignKey, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.exc import IntegrityError

Base = declarative_base()

class Athlete(Base):
    __tablename__ = 'athletes'
    
    athlete_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    gender = Column(String)

class Event(Base):
    __tablename__ = 'events'
    
    event_id = Column(Integer, primary_key=True, autoincrement=True)
    location = Column(String)
    year = Column(Integer)
    distance = Column(Integer)
    stroke = Column(String)
    is_relay = Column(Boolean)
    nb_relay = Column(Integer, nullable=True)

class NationalTeam(Base):
    __tablename__ = 'national_teams'
    
    national_team_id = Column(Integer, primary_key=True, autoincrement=True)
    code = Column(String)

class EventTeam(Base):
    __tablename__ = 'event_teams'
    
    event_team_id = Column(Integer, primary_key=True, autoincrement=True)
    event_id = Column(Integer, ForeignKey('events.event_id'))
    athlete_id = Column(Integer, ForeignKey('athletes.athlete_id'))
    national_team_id = Column(Integer, ForeignKey('national_teams.national_team_id'))

class Result(Base):
    __tablename__ = 'results'
    
    result_id = Column(Integer, primary_key=True, autoincrement=True)
    event_team_id = Column(Integer, ForeignKey('event_teams.event_team_id'))
    results = Column(Float, nullable=True)
    rank = Column(Integer)
    quit_reason = Column(String, nullable=True)

# Création des tables si elles n'existent pas déjà
try:
    Base.metadata.create_all(engine)
except IntegrityError:
    print("Les tables existent déjà.")

Enfin on peuple les tables avec les données.

In [ ]:

from sqlalchemy.orm import sessionmaker
import pandas as pd

# Création d'une session
Session = sessionmaker(bind=engine)
session = Session()

# Fonctions pour ajouter ou récupérer des données
def get_or_create_athlete(name, gender):
    athlete = session.query(Athlete).filter_by(name=name, gender=gender).first()
    if not athlete:
        athlete = Athlete(name=name, gender=gender)
        session.add(athlete)
        session.commit()
    return athlete

def get_or_create_event(location, year, distance, stroke, is_relay, nb_relay):
    nb_relay = None if is_relay == False else int(nb_relay)
    event = session.query(Event).filter_by(location=location, year=year, distance=distance, stroke=stroke, is_relay=is_relay, nb_relay=nb_relay).first()
    if not event:
        event = Event(location=location, year=year, distance=distance, stroke=stroke, is_relay=is_relay, nb_relay=nb_relay)
        session.add(event)
        session.commit()
    return event

def get_or_create_national_team(code):
    team = session.query(NationalTeam).filter_by(code=code).first()
    if not team:
        team = NationalTeam(code=code)
        session.add(team)
        session.commit()
    return team

def get_or_create_team(event_id, athlete_id, national_team_id):
    team = session.query(EventTeam).filter_by(event_id=event_id, athlete_id=athlete_id, national_team_id=national_team_id).first()
    if not team:
        team = EventTeam(event_id=event_id, athlete_id=athlete_id, national_team_id=national_team_id)
        session.add(team)
        session.commit()
    return team

def get_or_create_result(event_team_id, results, rank, quit_reason):
    result = session.query(Result).filter_by(event_team_id=event_team_id, results=results, rank=rank).first()
    if not result:
        result = Result(event_team_id=event_team_id, results=results, rank=rank, quit_reason=quit_reason)
        session.add(result)
    return result

# Parcourir le dataframe et ajouter les données
for _, row in df_relay_expanded.iterrows():
    athlete = get_or_create_athlete(row['athlete'], row['gender'])
    national_team = get_or_create_national_team(row['team'])
    event = get_or_create_event(row['location'], row['year'], row['distance'], row['stroke'], row['is_relay'], row['nb_relay'])
    event_team = get_or_create_team(event.event_id, athlete.athlete_id, national_team.national_team_id)
    get_or_create_result(event_team.event_team_id, row['results'], row['rank'], row.get('quit_reason'))

# Commit final pour sauvegarder toutes les modifications
session.commit()

# Fermer la session
session.close()

print("Base de données peuplée avec succès!")

Je vais juste convertir les NULL dans nb_relay en 1, je vais pas refactoriser, juste en executant la cell ça remplacera les valeurs.

In [ ]:
# Remplacer les valeurs NULL de nb_relay par 1 dans la table events
from sqlalchemy import update
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

stmt = update(Event).where(Event.nb_relay.is_(None)).values(nb_relay=1)
session.execute(stmt)
session.commit()

session.close()